# 전자결재 문서 Export v2

DB `documents` 테이블을 조회하여 `addDocument` 명령어 형식의 `.cmds` 파일로 내보내기

**주요 기능:**
- 3가지 모드 지원: 전체(all), 연도별(year), 특정 source_id(source_ids)
- sourceId 형식: `doc_{source_id}_{시도횟수}` 형태로 출력
- source_id 기준 오름차순 정렬
- JSON 파싱 실패 시 경고 출력

## 1. 설정

In [1]:
import pymysql
import json

# ============ DB 설정 ============
USER = "root"
PASS = "1234"
HOST = "127.0.0.1"
PORT = 3306
DB = "any_approval"

# ============ sourceId 형식 설정 ============
# doc_{source_id}_{ATTEMPT_NUMBER} 형태로 출력됨
# 예: ATTEMPT_NUMBER = 6 → "doc_15241300_06"
ATTEMPT_NUMBER = 9

# ============ 모드 선택 (하나만 사용) ============
# "all"        : 전체 문서
# "year"       : 특정 연도 (단일 또는 복수)
# "source_ids" : 특정 source_id 목록
MODE = "source_ids"

# ============ 모드별 파라미터 ============
# year 모드: 단일 연도 또는 리스트
# 예: TARGET_YEAR = 2024 또는 TARGET_YEAR = [2023, 2024, 2025]
TARGET_YEAR = [2024, 2025]

# source_ids 모드: source_id 리스트
TARGET_SOURCE_IDS = ["2002144", "2002200", "2002203", "2003868", "2005560", "2005586", "2005627", "2006643", "2008527", "2008535", "2008630", "2008724", "2008796", "2008809", "2008862", "2008997", "2009051", "2009076", "2009180", "2009192", "2009237", "2009376", "2009390", "2009490", "2009582", "2009619", "2009706", "2993923", "3008052", "3105428", "3185978", "3244536", "5838445", "5908504", "6335236", "6648935", "7285423", "7708182", "7936187", "8210102", "8696529", "10199204", "10731264", "11082135", "11213102", "12412844", "12446706", "12532840", "12596888", "12678178", "12678204", "12707319", "12708102", "12754985", "12782722", "12794668", "12939963", "13162037", "13166906", "13166926", "13166974", "13180565", "13198860", "13201458", "13214738", "13299088", "13300483", "13300558", "13301010", "13308312", "13337725", "13349629", "13475914", "13475923", "13487814", "13487830", "13511036", "13613684", "13648901", "13734864", "13790114", "13813156", "16161555", "16307793", "16307841"]

# ============ 출력 파일명 ============
OUTPUT_FILE = "documents_09.cmds"

## 2. 유틸리티 함수

In [2]:
# 파싱 실패 카운트 추적용
parse_errors = []

def safe_json_loads(text_data, field_name="unknown", doc_id=None):
    """안전하게 JSON 파싱 (실패 시 경고 출력)"""
    if text_data is None:
        return []
    
    if isinstance(text_data, str):
        text_data = text_data.strip()
        if not text_data or text_data == '[]':
            return []
    
    try:
        return json.loads(text_data)
    except json.JSONDecodeError as e:
        error_info = {
            "doc_id": doc_id,
            "field": field_name,
            "error": str(e),
            "value_preview": str(text_data)[:100]
        }
        parse_errors.append(error_info)
        print(f"⚠️ JSON 파싱 실패 [doc_id={doc_id}, field={field_name}]: {e}")
        print(f"   값 미리보기: {str(text_data)[:100]}...")
        return []
    except Exception as e:
        error_info = {
            "doc_id": doc_id,
            "field": field_name,
            "error": str(e),
            "value_preview": str(text_data)[:100]
        }
        parse_errors.append(error_info)
        print(f"⚠️ 예상치 못한 파싱 오류 [doc_id={doc_id}, field={field_name}]: {e}")
        return []

## 3. DB 연결 및 데이터 조회

In [3]:
# DB 연결
conn = pymysql.connect(
    host=HOST,
    port=PORT,
    user=USER,
    password=PASS,
    database=DB,
    charset='utf8mb4'
)

cursor = conn.cursor(pymysql.cursors.DictCursor)

# 기본 SELECT 절
base_select = """
SELECT id, source_id, doc_num, `references`, doc_type, title, `attaches`, 
       drafter_name, drafter_position, drafter_dept, drafter_email, drafter_dept_code,
       created_at, created_date, doc_body, doc_status, `referrers`, activities, 
       form_name, is_public, end_year
FROM documents
"""

# 모드에 따라 쿼리 생성
if MODE == "all":
    query = base_select + "ORDER BY CAST(source_id AS UNSIGNED) ASC"
    print(f"📌 모드: 전체 문서")

elif MODE == "year":
    # 단일 연도 또는 리스트 처리
    if isinstance(TARGET_YEAR, list):
        years_str = ", ".join(str(y) for y in TARGET_YEAR)
        query = base_select + f"WHERE year IN ({years_str}) ORDER BY CAST(source_id AS UNSIGNED) ASC"
        print(f"📌 모드: 연도별 (year IN ({years_str}))")
    else:
        query = base_select + f"WHERE year = {TARGET_YEAR} ORDER BY CAST(source_id AS UNSIGNED) ASC"
        print(f"📌 모드: 연도별 (year = {TARGET_YEAR})")

elif MODE == "source_ids":
    ids_str = ", ".join(f"'{sid}'" for sid in TARGET_SOURCE_IDS)
    query = base_select + f"WHERE source_id IN ({ids_str}) ORDER BY CAST(source_id AS UNSIGNED) ASC"
    print(f"📌 모드: 특정 source_id ({len(TARGET_SOURCE_IDS)}건)")

else:
    raise ValueError(f"잘못된 MODE 값: {MODE}. 'all', 'year', 'source_ids' 중 하나를 선택하세요.")

# 쿼리 실행
cursor.execute(query)
rows = cursor.fetchall()

print(f"📊 총 {len(rows):,}개 문서 조회 완료")

📌 모드: 특정 source_id (85건)
📊 총 85개 문서 조회 완료


## 4. 문서 변환 및 파일 생성

In [4]:
# 에러 카운트 초기화
parse_errors.clear()

with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    for idx, row in enumerate(rows):
        doc_id = row['id']
        
        # sourceId 형식: doc_{source_id}_{ATTEMPT_NUMBER:02d}
        formatted_source_id = f"doc_{row['source_id']}_{ATTEMPT_NUMBER:02d}"
        
        # JSON 필드 파싱
        references_json = safe_json_loads(row['references'], 'references', doc_id)
        attaches_json = safe_json_loads(row['attaches'], 'attaches', doc_id)
        referrers_json = safe_json_loads(row['referrers'], 'referrers', doc_id)
        activities_json = safe_json_loads(row['activities'], 'activities', doc_id)
        
        # JSON 구조 생성
        doc = {
            "sourceId": formatted_source_id,
            "docNum": row['doc_num'],  # 원본 그대로 (공백 유지)
            "references": references_json,
            "docType": row['doc_type'],
            "title": row['title'],
            "attaches": attaches_json,
            "drafter": {
                "positionName": row['drafter_position'],
                "deptName": row['drafter_dept'],
                "name": row['drafter_name'],
                "emailId": row['drafter_email'],
                "deptCode": row['drafter_dept_code']
            },
            "createdAt": row['created_at'],
            "docBody": row['doc_body'],
            "docStatus": row['doc_status'],
            "referrers": referrers_json,
            "activities": activities_json,
            "formName": row['form_name'],
            "isPublic": bool(row['is_public'])
        }
        
        # addDocument 형식으로 작성
        f.write(f"addDocument {json.dumps(doc, ensure_ascii=False, separators=(',', ':'))}\n")
        
        # 진행 상황 표시 (1000개마다)
        if (idx + 1) % 1000 == 0:
            print(f"⏳ {idx + 1:,}개 처리 완료...")

print(f"\n✅ 총 {len(rows):,}개 문서를 {OUTPUT_FILE} 파일로 저장했습니다.")


✅ 총 85개 문서를 documents_09.cmds 파일로 저장했습니다.


## 5. 파싱 에러 요약

In [5]:
if parse_errors:
    print(f"\n📋 JSON 파싱 에러 요약: 총 {len(parse_errors)}건")
    print("-" * 60)
    
    # 필드별 에러 카운트
    from collections import Counter
    field_counts = Counter(e['field'] for e in parse_errors)
    
    print("\n[필드별 에러 건수]")
    for field, count in field_counts.most_common():
        print(f"  - {field}: {count}건")
    
    print("\n[에러 상세 목록 (처음 10건)]")
    for err in parse_errors[:10]:
        print(f"  doc_id={err['doc_id']}, field={err['field']}")
        print(f"    error: {err['error']}")
        print(f"    preview: {err['value_preview']}")
        print()
else:
    print("\n✅ JSON 파싱 에러 없음!")


✅ JSON 파싱 에러 없음!


## 6. 결과 미리보기 (선택)

In [6]:
# 생성된 파일의 처음 5줄 미리보기
print(f"📄 {OUTPUT_FILE} 미리보기 (처음 5줄):\n")
with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= 5:
            break
        # 너무 길면 잘라서 표시
        preview = line[:150] + "..." if len(line) > 150 else line
        print(f"{i+1}: {preview}")

📄 documents_09.cmds 미리보기 (처음 5줄):

1: addDocument {"sourceId":"doc_2002144_09","docNum":"ANY-사업- 0012","references":[],"docType":"DRAFT","title":"한국한의학연구원_변경CP 품의 바랍니다","attaches":[{"name"...
2: addDocument {"sourceId":"doc_2002200_09","docNum":"ANY-사업- 0023","references":[],"docType":"DRAFT","title":"한국한의학연구원 변경CP 품의 요청 (20100310)","attaches"...
3: addDocument {"sourceId":"doc_2002203_09","docNum":"ANY-경비- 0024","references":[],"docType":"DRAFT","title":"한의학연구원_PM교체에따른_3월12일까지 법인카드사용내역서 보고","atta...
4: addDocument {"sourceId":"doc_2003868_09","docNum":"ANY-경비- 0450","references":[],"docType":"DRAFT","title":"삼성건설_2012년 01월 체크카드사용내역(재기안)","attaches":[...
5: addDocument {"sourceId":"doc_2005560_09","docNum":"ANY-경비- 0014","references":[],"docType":"DRAFT","title":"5월 야근식대 품의","attaches":[{"name":"IMG_20130...


## 7. 연결 종료

In [7]:
cursor.close()
conn.close()
print("🔌 DB 연결 종료")

🔌 DB 연결 종료
